In [12]:
import binance
import pandas as pd
import numpy as np
import math

In [4]:
ticker = "MANAUSDT"
client = binance.Client()

# Traemos precio actual

In [5]:
precio_actual = float(client.futures_mark_price(symbol=ticker)['markPrice'])
precio_actual

0.966

# Traemos el Order Book de la moneda

In [6]:
order_book = client.futures_order_book(symbol=ticker, limit=1000)
order_book

{'lastUpdateId': 1769855412025,
 'E': 1659409337649,
 'T': 1659409337631,
 'bids': [['0.9660', '40'],
  ['0.9659', '471'],
  ['0.9658', '1421'],
  ['0.9657', '8527'],
  ['0.9656', '2377'],
  ['0.9655', '1673'],
  ['0.9654', '932'],
  ['0.9653', '4814'],
  ['0.9652', '4990'],
  ['0.9651', '6964'],
  ['0.9650', '2653'],
  ['0.9649', '8010'],
  ['0.9648', '15255'],
  ['0.9647', '2672'],
  ['0.9646', '8610'],
  ['0.9645', '4275'],
  ['0.9644', '8290'],
  ['0.9643', '7260'],
  ['0.9642', '6247'],
  ['0.9641', '9506'],
  ['0.9640', '7001'],
  ['0.9639', '10340'],
  ['0.9638', '4228'],
  ['0.9637', '6498'],
  ['0.9636', '11160'],
  ['0.9635', '2135'],
  ['0.9634', '3625'],
  ['0.9633', '10127'],
  ['0.9632', '1469'],
  ['0.9631', '6856'],
  ['0.9630', '7713'],
  ['0.9629', '11240'],
  ['0.9628', '23190'],
  ['0.9627', '8837'],
  ['0.9626', '2247'],
  ['0.9625', '3174'],
  ['0.9624', '3796'],
  ['0.9623', '12281'],
  ['0.9622', '28166'],
  ['0.9621', '3860'],
  ['0.9620', '8323'],
  ['0.9619',

# Extraer ventas y compras

## Ventas

- Del order book extraemos las ventas (asks) y lo pasamos a un dataframe
- Para dejarlo a cómo lo muestra el libro, las ventas las ordenamos de mayor a menor
- Para poder trabajar los datos los pasamos a números, en este caso float

In [7]:
ventas = order_book['asks']
ventas = pd.DataFrame(ventas, columns = ['precio', 'monedas']).sort_index(ascending=False)
ventas[['precio','monedas']] = ventas[['precio','monedas']].astype(float)
ventas

,precio,monedas
999,1.0953,28.0
998,1.0951,269.0
997,1.0950,2069.0
996,1.0949,13.0
995,1.0948,6.0
...,...,...
4,0.9665,10916.0
3,0.9664,6827.0
2,0.9663,4419.0
1,0.9662,3633.0


## Compras

- Hacemos lo mismo que ventas, convertimos a float las columnas de precio y monedas

In [8]:
compras = order_book['bids']
compras = pd.DataFrame(compras, columns=['precio','monedas'])
compras[['precio','monedas']] = compras[['precio','monedas']].astype(float)
compras

,precio,monedas
0,0.9660,40.0
1,0.9659,471.0
2,0.9658,1421.0
3,0.9657,8527.0
4,0.9656,2377.0
...,...,...
995,0.8492,127.0
996,0.8491,37.0
997,0.8490,292.0
998,0.8488,11.0


# Creación de rangos

- Creamos un rango tanto para ventas como compras 
- Este rango es derivado del precio máximo y mínimo

## Obtenemos máximos y mínimos para las ventas y compras

In [9]:
ventas_max = ventas['precio'].max()
print(ventas_max)

ventas_min = ventas['precio'].min()
print(ventas_min)

1.0953
0.9661


In [10]:
compras_max = compras['precio'].max()
print(compras_max)
compras_min = compras['precio'].min()
print(compras_min)

0.966
0.8486


- Creamos rangos con esos valores
- Para esto necesitamos crear un diccionario donde coloquemos, para cada moneda, los múltiplos para saber los incrementos del rango
- Recordando que vamos a tratar de capturar todos los rangos permitidos del libro de binance

In [11]:
dic = {
    'MANAUSDT': {
        'd': 4,
        'i1': 0.01,
        'i2': 0.001,
        'i3': 0.0001
    }
}

- Ahora obtenemos el inicio del rango , usando:

(precio min / dic['ticker'] ) -> De este número, nos interesa el entero inferior. Por lo que lo redondearemos hacia el entero inferior (math.floor())

math.floor((precio min / dic['ticker'] )) * dic['ticker'] -> Luego multplicamos por el mismo incremento

- Ahora obtenemos el final del rango , usando:

(precio max / dic['ticker'] ) -> De este número, nos interesa el entero superior. Por lo que lo redondearemos hacia el entero superior (math.ceil())

math.ceil( (precio max / dic['ticker'] ) ) * dic['ticker']

In [21]:
rango_v_i = math.floor(ventas_min / dic[ticker]['i1']) * dic[ticker]['i1']
rango_v_i
rango_v_f = math.ceil(ventas_max / dic[ticker]['i1']) * dic[ticker]['i1']
rango_v_f
rango_v = np.arange(rango_v_i, rango_v_f, dic[ticker]['i1'])

array([0.96, 0.97, 0.98, 0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06,
       1.07, 1.08, 1.09, 1.1 ])

In [52]:
rango_c_i = math.floor(compras_min / dic[ticker]['i1']) * dic[ticker]['i1']
rango_c_i
rango_c_f = (math.ceil(compras_max / dic[ticker]['i1']) * dic[ticker]['i1'])
rango_c_f = rango_c_f + dic[ticker]['i1']
rango_c = np.arange(rango_c_i, rango_c_f, dic[ticker]['i1'])

In [77]:
#compras
#rango_c
compras_ag = compras.groupby(pd.qcut(compras.precio, rango_c)).sum()
#print(compras_ag['monedas'].max())

idx = compras_ag.monedas.idxmax()
idx.left
print(f'Máx compras: {compras_ag["monedas"].max()}')
print(f'En el índice: {compras_ag.monedas.idxmax()}')
print(f'Precio: ${idx.right}')
#compras_ag = compras

Máx compras: 185207.0
En el índice: (0.959, 0.96]
Precio: $0.96


In [ ]:
#compras
#rango_c
ventas_ag = ventas.groupby(pd.qcut(ventas.precio, rango_c)).sum()
#print(compras_ag['monedas'].max())

idx = compras_ag.monedas.idxmax()
idx.left
print(f'Máx compras: {compras_ag["monedas"].max()}')
print(f'En el índice: {compras_ag.monedas.idxmax()}')
print(f'Precio: ${idx.right}')